<a href="https://colab.research.google.com/github/Adrianxwu/MDO-ML-IVC-ITB-2021/blob/main/Tutorial_7_Surrogate_Modelling_for_Students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TUTORIAL 7: SURROGATE MODELLING AND MACHINE LEARNING: MODERN APPROXIMATION TOOLS**

**3 August 2021**

<img src="https://mdoml2021.ftmd.itb.ac.id/wp-content/uploads/2021/06/image.png" width="15%"><br>
<small>2021 © MDOML IVC ITB</small>


Based on the lecture material by: Pramudita Satria Palar, Ph.D. (Institut Teknologi Bandung, Indonesia)

Please run the following code first to import all required packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import itertools
import pandas as pd
from google.colab import files

# Introduction
The task of any surrogate model is to approximate an unknown black box function $f(\mathbf{x})$, where $\mathbf{x}=\{x_{1},x_{2},\ldots,x_{m}\}$ and $m$ is the dimensionality of the input variable, with an approximation function $\hat{f}(\mathbf{x})$. The approximation function can take any form, e.g. polynomial or radial basis function. 

First, we need to prepare the design of experiment (DoE) consisting of $n$ sampling points, i.e., $\mathcal{X}=\{\mathbf{x}^{(1)},\mathbf{x}^{(2)},\ldots,\mathbf{x}^{(n)}\}^{T}$. The responses of $\mathcal{X}$ can be calculated by physical or computer experiments to yield $\mathbf{y}=\{y^{(1)},y^{(2)},\ldots,y^{(n)}\}^{T} = \{f(\mathbf{x}^{(1)}),f(\mathbf{x}^{(2)}),\ldots,f(\mathbf{x}^{(n)})\}^{T}$. There are several DoE technique that we can use to generate $\mathcal{X}$ including latin hypercube sampling, Sobol sequence, and Halton sequence.

A good surrogate model should approximate the true function well, i.e., $f(\mathbf{x})\approx \hat{f}(\mathbf{x})$. However, depending on the application, sometimes the surrogate is only required to be accurate on interesting regions (e.g. accurate in the basin of global optimum). 

In this tutorial, we will learn the basic concepts of two surrogate models, namely, **polynomial regression (PR)** and **radial basis function (RBF)**. We think that the aforementioned methods are suitable for educational purpose for the students to study other, possibly more advanced, surrogate models. 


# Generalized Linear Model
Both PR and RBF are **generalized linear model**, that is

> $\hat{f}(\mathbf{x})=\boldsymbol{\alpha}^{T}\boldsymbol{\varphi}=\sum_{i=1}^{p}\alpha_{i} \phi_{i}(\mathbf{x})$

where $\boldsymbol{\alpha}=\{\alpha_{1},\alpha_{2},\ldots,\alpha_{p}\}^{T}$ is the vector of coefficients, $p$ is the size of basis functions set, and $\boldsymbol{\varphi}$ is the vector of basis functions. The basis functions $\boldsymbol{\varphi}$ differ for PR and RBF, as will be discussed next. Estimation of $\boldsymbol{\alpha}$ can be done by various means such as by using least-squares or maximizing the model likelihood. In both RBF and PR, $\boldsymbol{\alpha}$  is obtained by solving a system of linear equations.

# Latin hypercube sampling
Through this tutorial, we will use the latin hypercube sampling (LHS) method to generate the sampling points. The function is ```sampling_rlh(Nsamp=, dimen=, edges=)```, where ```Nsamp``` is the number of samples, ```dimen``` is the input dimensionality, and ```edges``` determines whether we want the outermost samples to be located on the bounds or not (i.e, set ```edges=1``` if you want to do so and ```edges=0``` otherwise) 

 as defined in the following cell:

In [ ]:
def sampling_rlh(
    Nsamp, dimen, edges = 0
):

  X = np.zeros(shape=[Nsamp, dimen])

  for i in range(dimen):
     X[:, i] = np.random.permutation(Nsamp) + 1

  if edges == 1:
      X = (X - 1)/(Nsamp -1)
  else:
      X = (X - 0.5)/Nsamp

  return X



Let's try to generate an example of sampling points based on LHS:

In [ ]:
Xex = sampling_rlh(Nsamp=20,dimen=2,edges=0) # An example of LHS-generated sampling points

# Plotting
plt.scatter(Xex[:,0],Xex[:,1],s=20,c='b')
plt.xlabel('$x_{1}$')
plt.ylabel('$x_{2}$')
plt.show()

# Polynomial Regression

The standard PR uses monomial basis function in the formulation (i.e., $x,x^{2},x^{3},\ldots$), which is something that you are already familiar with. In this tutorial, we will extend the one-dimensional PR into multidimensional approximation with $m$ variables. 

In this paper, we will study polynomial regression through the polynomial chaos expansion (PCE) method. Notice that there exists an intrusive version of PCE (hence, it is not a surrogate model). 

> $\hat{f}(\mathbf{x})=\sum_{j=0}^{p} \alpha_{j} \Psi_{j}(\mathbf{x})$

where $\boldsymbol{\alpha}$ is now the vector of PCE coefficients and $\Psi$ is a multidimensional orthogonal polynomial basis. Notice that the index starts from zero for convenience since $\Psi_{0}(\mathbf{x})$. The nice thing about PCe is that several quantities of interests such as Sobol indices, mean, and variance of the output can be directly calculated from the coefficients. Orthogonal polynomials are polynomials that satisfy the following relation:
> $\int_{\Omega}\Psi_{i}(\mathbf{x})\Psi_{j}(\mathbf{x}) d \mathbf{x} = \delta_{ij}$
 
where $\delta_{ij}$ is the kronecker delta (i.e., $\delta_{ij}=1$ if $i=j$ and $\delta_{ij}=0$ if $i \neq j$).

The coefficients $\boldsymbol{\alpha}$ can be obtained by solving the following system of linear equation:
> $\boldsymbol{F} \boldsymbol{\alpha} = \mathbf{y}$

where $F$ is an $n \times p $ regression matrix with its $(i,j)$-th component is $F_{i,j} = \Psi_{j}(\mathbf{x}^{(i)})$. Thus, $\boldsymbol{\alpha}=\boldsymbol{F}^{-1} \mathbf{y}$. Notice that the regression matrix is constructed by using the experimental design $\mathcal{X}$ and the defined polynomial basis set.

In this tutorial, we will use Legendre polynomials in our PCE. The reason why Legendre polynomials are used in this paper is due to the bounded input space. 

In practice, it is more often that we have $\mathcal{X}$ first and we need to define the most suitable polynomial basis set. We will use LOOCV to find the polynomial basis set that yields the lowest CV error. 


---
## Polynomial Indexing

While it is not obvious from the theoritical explanation, the code implementation for each polynomial terms is a bit complex.

The output of this polynomial indexing problem is to create the following matrix:

$ \begin{pmatrix} q^{1}_{0} & ... & q^{m}_{0} \\\ \vdots &  & \vdots \\\ q^{1}_{p} & ... & q^{m}_{p} \end{pmatrix}$

where $q^{i}_{j}$ is the power of the polynomial term in the $i$-th dimension and $j$-th polynomial basis.

The cell below is the implementation on how to index all the polynomial terms. The main subroutine that we will use is ```total_trunc(pmax, dimen)```, which generates polynomial indices based on **total order truncation**.

In [ ]:
#@title Generating polynomial indices
def nchoosek(n, k):
    C = itertools.combinations(n, k)
    C = np.array(list(C))
      
    return C

def repmat(A, rowx, colx):
    row = A.shape[0]
    col = A.shape[1]
    X = np.zeros(shape=[row*rowx, col*colx])
    Xcol = np.zeros(shape=[row, col*colx])

    for i in range(colx):
      Xcol[:, (col*i):(col*(i+1))] = A

    for i in range(rowx):
      X[(row*i):(row*(i+1)), :] = Xcol

    return X

def index_num(u, v, k):
    rowu = u.shape[0]
    colu = u.shape[1]
    ind_u = []
    c = 0
    for j in range(colu):
      for i in range(rowu):
        if u[i, j] == k :
          temp_u = [i, j] 
          ind_u.append(temp_u)
          c = c+1

    uv = np.zeros(shape=[c])
    for i in range(c):
      temp_u = ind_u[i]
      uv[i] = v[temp_u[0], temp_u[1]]

    return uv

def mon_cof(pmax, dimen):
    nn = pmax+dimen-1
    v = np.zeros(shape=[nn])
    for i in range(nn):
      v[i] = i+1

    c = nchoosek(v, dimen-1)
    m = c.shape[0]
    t = np.ones(shape=[m, nn])  
    m_in = np.zeros(shape=[m])
    for i in range(m):
      m_in[i] = i+1

    m_in = np.transpose(m_in)
    c_in = (c-1)*m
    m_in = m_in.reshape(m_in.shape[0], 1)
    t_in = repmat(m_in, 1, dimen-1)
    t_in = t_in.reshape(c_in.shape[0], c_in.shape[1])
    t_in = t_in + c_in 
    for i in range(t_in.shape[0]):
      for j in range(t_in.shape[1]):
        ind = t_in[i,j]
        it = int(ind % m) -1
        jt = int(np.ceil(ind/m)) -1
        #print(it, jt)
        t[it, jt] = 0

    u = np.zeros(shape=[1, m])
    u = np.concatenate((u, t.T))
    u = np.concatenate((u, np.zeros(shape=[1, m])))
    v = np.cumsum(u, axis = 0)
    uv0 = index_num(u, v, 0)
    uv = uv0.reshape(m, dimen+1)
    uv = uv.T  
    X = np.zeros(shape=[uv.shape[0]-1, m])  
    for i in range(uv.shape[0]-1):
      X[i, :] = uv[i+1, :] - uv[i, :]

    X = X.T
    return X

def total_trunc(pmax, dimen):
    for i in range(pmax, -1, -1):
      if i == pmax:
        idx = mon_cof(i, dimen)
      else:
        idx = np.concatenate((idx, mon_cof(i, dimen)))
    idx = idx[::-1,:]

    return idx

You can try generating the indices polynomial basis by executing the cell below

In [ ]:
index = total_trunc(pmax=3, dimen=2)

# Plotting
plt.scatter(index[:,0],index[:,1],s=100,c='b')
plt.xlabel('$x_{1}$')
plt.ylabel('$x_{2}$')
plt.show()

print(index)

---
## Polynomial Basis

After we succesfully assign indices to all polynomial terms, the rest of the codes are straight from the theoritical definitions. Basically, we need to generate the regression matrix $\boldsymbol{F}$ once we define the polynomial bases and the experimental design. Our code below (i.e., ```create_poly```) generates the regression matrix by taking polynomial indices, normalized sampling points, and polynomial type (i.e., monomial or Legendre). We also the Legendre polynomial as a recursive function.

In [ ]:
def create_poly(
    Id_pol, Xnorm, poly_type
):

    Nsamp = Xnorm.shape[0]
    dimen = Xnorm.shape[1]
    nbase = Id_pol.shape[0]
    F = np.zeros(shape=[Nsamp, nbase])

    if poly_type in 'monomial':
      for i in range(Nsamp):
        for j in range(nbase):
          temp = 1
          for k in range(dimen):
            temp = temp * Xnorm[i, k]**(Id_pol[j, k])
          F[i, j] = temp
      
    elif poly_type in 'legendre':    
      for i in range(Nsamp):
        for j in range(nbase):
          temp = 1
          for k in range(dimen):
            #temp = temp * spc.eval_legendre(Id_pol[j, k], Xnorm[i, k])
            temp = temp * legendre_eval(Id_pol[j, k], Xnorm[i, k])         

          F[i, j] = temp


    return F   

def legendre_eval(order, x):
  if int(order) == 0:
    fx = 1
  elif int(order) == 1:
    fx = x
  elif order > 1:
    fx = (2*order - 1)/order * x * legendre_eval(order-1, x) - (order - 1)/order * legendre_eval(order - 2, x)

  return fx

The following cell is the subroutine for predicting the output at an arbitrary point:

In [ ]:
def pred_PR(
    Xpred, coef, Id_pol, poly_type
):
  Npred = Xpred.shape[0]
  dimen = X.shape[1]
  Xnpred = np.zeros(shape = [Npred, dimen])
  
  for i in range(dimen):
      Xnpred[:, i] = (Xpred[:, i] - bounds[0, i])/(bounds[1, i] - bounds[0, i])

  if poly_type in 'legendre':
     for i in range(dimen):
        Xnpred[:, i] = 2*(Xnpred[:, i] - 0.5)  

  pred_pol = create_poly(Id_pol, Xnpred, poly_type)
  y = pred_pol@coef
  return y

We also define the following lines of code for calculating the LOOCV error:

In [ ]:
def LOOCV_PR(Id_pol, Xnorm, y, poly_type):
  Nsamp = Xnorm.shape[0]
  Ypred = np.zeros(shape=[Nsamp])
  dimen = Xnorm.shape[1]

  for i in range(Nsamp):
    Xnpred = Xnorm[i, :].reshape(1, dimen)
    Xtemp = np.delete(Xnorm, i, 0)
    Ytemp = np.delete(y, i)

    temp_poly = create_poly(Id_pol, Xtemp, poly_type)
    term1 = temp_poly.T @ Ytemp
    term2 = temp_poly.T @ temp_poly

    coef = np.linalg.solve(term2, term1)
    Ypred[i] = pred_PR_LOO(Xnpred, coef, Id_pol, poly_type)

  rmse = np.sqrt(np.sum((Ypred - y)**2)/Nsamp)

  return rmse

def pred_PR_LOO(
    Xnpred, coef, Id_pol, poly_type
):
  pred_pol = create_poly(Id_pol, Xnpred, poly_type)
  y = pred_pol@coef
  return y

---
## Main polynomial regression routine

All that's left is to set up the main routine for the polynomial regression. The main routine is written in the following cell. Note that the code below implements automatic selection of polynomial order; you only need to define the maximum order (i.e., ```max_order```). Furthermore, you also need to define the type of polynomials, that is, either ```monomial``` or ```Legendre```.

In [ ]:
def main_PR(
    X, y, bounds, max_order, poly_type = 'monomial'
):

  #Take all the required parameters
  class result:
    def __init__ (self):
      self.coef = coef
      self.residual = resd_all[id_min]
      self.polynom = poly_all[id_min]
      self.Id_pol = Id_all[id_min]
      self.Id_pred = Id_pred
      self.coef_all = coef_all
      self.resd_all = resd_all
      self.poly_all = poly_all
      self.Id_all = Id_all
      self.bounds = bounds
      self.Xnorm = Xnorm
      self.X = X
      self.y = y
      self.best_order = best_order
      self.poly_type = poly_type
      self.LOOCV_all = LOOCV_all
      self.LOOCV = LOOCV_all[id_min]

  # Normalize X into [-1, 1]
  Nsamp = X.shape[0]
  dimen = X.shape[1]
  Xnorm = np.zeros(shape = [Nsamp, dimen])

  for i in range(dimen):
    Xnorm[:, i] = (X[:, i] - bounds[0, i])/(bounds[1, i] - bounds[0, i])

  if poly_type in 'legendre':
    for i in range(dimen):
      Xnorm[:, i] = 2*(Xnorm[:, i] - 0.5)

  # do loop for all orders
  coef_all = []
  resd_all = []
  poly_all = []
  Id_all = []
  LOOCV_all = []

  for i in range(max_order):
    current_order = i+1
    Id_pol = total_trunc(current_order, dimen)
    nbase = Id_pol.shape[0]
    if nbase > Nsamp:
      raise NameError('Number of basis is higher than number of samples')

    F = create_poly(Id_pol, Xnorm, poly_type)

    #Find the optimum coefficient through LOOCV function

    term1 = F.T @ y
    term2 = F.T @ F
    opt_coef = np.linalg.solve(term2, term1)

    current_resd = residual_ls(opt_coef, y, F)
    current_LOOCV = LOOCV_PR(Id_pol, Xnorm, y, poly_type)

    coef_all.append(opt_coef)
    resd_all.append(current_resd)
    poly_all.append(F)
    Id_all.append(Id_pol)
    LOOCV_all.append(current_LOOCV)

  #find model with lowest loocv
  id_min = LOOCV_all.index(min(LOOCV_all))
  coef = coef_all[id_min]
  best_order = id_min + 1
  Id_pred = total_trunc(best_order, dimen)

  return result()

def residual_ls(coef, y, M):
  pred = M @ coef 
  resd = np.sum((y - pred)**2)

  return resd


---
## Example 1: Monomial Polynomial Regression

The first test case for our polynomial regression model is a third order polynomial function with an extra sinusoidal term to make it more difficult, defined as

$f(\mathbf{x}) = x_{1}^2 + x_{1} + x_{2}^{3}-0.5\text{sin}(x_{2}\pi)$

where $x_{i} \in [-2,2]$.

 The following cell defines our test function:



In [ ]:
def fun2D_polynom(X):
  Nsamp = X.shape[0]
  y = 1*X[:, 0]**2 + X[:, 0] + X[:, 1]**3 - 0.5*np.sin(X[:, 1]*np.pi)

  y = y.reshape(Nsamp)

  return y
    


Now in this case, we are going to show the accuracy of polynomial regression model with a predefined maximum order. The main PR code will automatically select the best polynomial order that yields the lowest LOOCV error, which greatly reduces the overfitting risk. Please play around with the number of samples (```Nsamp```) and the maximum order (```max_order```) and observe how your prediction change.

In [ ]:
#2 dimensional example and 2 max order
dimen = 2 # Input dimensionality
Nsamp = 30# Number of sample
max_order = 2 # Maximum polynomial order

lb = np.array([-2, -2]) # Lower bounds
ub = np.array([2, 2]) # Upper bounds
bounds = np.array([lb, ub]) # Lower and upper bounds

# Sampling points generation
X = sampling_rlh(Nsamp, dimen) # Generate sampling points
X = X*(bounds[1, :] - bounds[0, :]) + bounds[0, :] # Convert the bounds into the real scale
y = fun2D_polynom(X) # evaluate the function

# Construct PR model
myPR = main_PR(X, y, bounds, max_order, poly_type = 'monomial')

Please plot the results by executing the cell below:

In [ ]:
#@title Polynomial regression prediction on a two-dimensional function
# For plotting
numplot = 50
Xpred = np.linspace(lb[0], ub[0], numplot)
Ypred = np.linspace(lb[1], ub[1], numplot)

Xplot, Yplot = np.meshgrid(Xpred, Ypred)
Zplot = np.zeros(shape=[numplot, numplot])
XYpred = np.zeros(shape=[numplot**2, dimen])
c = 0
for i in range(numplot):
    for j in range(numplot):
        XYpred[c, 0] = Xpred[i]
        XYpred[c, 1] = Ypred[j]
        Zplot[i, j] = fun2D_polynom(XYpred[c, :].reshape(1, dimen))
        c = c+1

ypred = pred_PR(XYpred, myPR.coef, myPR.Id_pred, myPR.poly_type)

Zpred = np.zeros(shape = [numplot, numplot])
c= 0
for i in range(numplot):
    for j in range(numplot):
        Zpred[i, j] = ypred[c]
        c = c+1    

plt.figure(0)
cf = plt.contourf(Xplot, Yplot, Zplot)
plt.scatter(X[:,0],X[:,1],s=60,c='r')
plt.colorbar()
plt.title("Real Surface")

plt.figure(1)
cf = plt.contourf(Xplot, Yplot, Zpred)
plt.scatter(X[:,0],X[:,1],s=60,c='r')
plt.colorbar()
plt.title("Polynomial Regression Surface")


You can check the polynomial order selected by the algorithm here:

In [ ]:
print("The polynomial order that yields the best LOOCV error is ",myPR.best_order)

The polynomial order that yields the best LOOCV error is  2


---
## Example 2: Polynomial Chaos Expansion with Legendre Polynomial

Please execute the following cell to test the polynomial regression with Legendre polynomials on the same two-dimensional function. We expect that there will be no significant difference between the predictive accuracy of PR and PCE for this problem. However, the differences may be more significant on other test functions.

In [ ]:
#2 dimensional example
dimen = 2 # Input dimensionality
Nsamp = 80 # Number of sample
max_order = 6 # Maximum polynomial order

lb = np.array([-2, -2]) # Lower bounds
ub = np.array([2, 2]) # Upper bounds 
bounds = np.array([lb, ub]) # Lower and upper bounds

# Sampling points generation
X = sampling_rlh(Nsamp, dimen) # Generate sampling points
X = X*(bounds[1, :] - bounds[0, :]) + bounds[0, :] # Convert the bounds into the real scale
y = fun2D_polynom(X) # Evaluate the function

# Construct PR and PCE model
myPR = main_PR(X, y, bounds, max_order, poly_type = 'legendre')
myPCE = main_PR(X, y, bounds, max_order, poly_type = 'monomial')

Please plot the results by executing the cell below:

In [ ]:
#@title PR and PCE prediction on a two-dimensional function

numplot = 50 
Xpred = np.linspace(lb[0], ub[0], numplot)
Ypred = np.linspace(lb[1], ub[1], numplot)

Xplot, Yplot = np.meshgrid(Xpred, Ypred)
Zplot = np.zeros(shape=[numplot, numplot])
XYpred = np.zeros(shape=[numplot**2, dimen])
c = 0

for i in range(numplot):
  for j in range(numplot):
    XYpred[c, 0] = Xpred[i]
    XYpred[c, 1] = Ypred[j]
    Zplot[i, j] = fun2D_polynom(XYpred[c, :].reshape(1, dimen))
    c = c+1

Zpred = np.zeros(shape = [numplot, numplot])
Zpred2 = np.zeros(shape= [numplot, numplot])
yval = np.zeros(shape = numplot**2)
yval2 = np.zeros(shape = numplot**2)

ypredPR = pred_PR(XYpred, myPR.coef, myPR.Id_pred, myPR.poly_type)
ypredPCE = pred_PR(XYpred, myPCE.coef, myPCE.Id_pred, myPCE.poly_type)

c= 0
for i in range(numplot):
  for j in range(numplot):
    Zpred[i, j] = ypredPR[c]
    Zpred2[i,j] = ypredPCE[c]
    yval[c] = (Zpred[i,j] - Zplot[i,j])**2
    yval2[c] = (Zpred2[i,j] - Zplot[i,j])**2
    c = c+1    

plt.figure(0)
cf = plt.contourf(Xplot, Yplot, Zplot)
plt.scatter(X[:,0],X[:,1],s=60,c='r')
plt.colorbar()
plt.title("Real Surface")

plt.figure(1)
cf = plt.contourf(Xplot, Yplot, Zpred)
plt.scatter(X[:,0],X[:,1],s=60,c='r')
plt.colorbar()
plt.title("Legendre Polynomial Regression Surface")

plt.figure(2)
cf = plt.contourf(Xplot, Yplot, Zpred2)
plt.scatter(X[:,0],X[:,1],s=60,c='r')
plt.colorbar()
plt.title("Ordinary Polynomial Regression Surface")

print("Averaged RMSE:")
print("Legendre polynomial")
print(np.sum(yval)/(yval.shape[0]))
print("Ordinary polynomial")
print(np.sum(yval2)/(yval.shape[0]))

---
# Radial Basis Function

Our second model is the Radial Basis Function (RBF) which is conceptually simpler than other kernel-based methods (e.g. support vector regression and Kriging) but also powerful. In contrast to PR that utilizes global basis functions, RBF employs more "localized" basis functions that are centered on the data points. 

The RBF model is expressed as follows

$\hat{f}(\mathbf{x}) = \boldsymbol{\alpha}^{T}\boldsymbol{\varphi} = \sum_{i=1}^{n} \alpha_{i} \varphi(\|\mathbf{x}-\mathbf{c}^{(i)} \|)$

where $\boldsymbol{\alpha}$ is the vector of RBF coefficients, $\boldsymbol{\varphi}$ is the set of RBFs/kernel functions, and $\mathbf{c}^{(i)}$ is the center of RBF (which is set as the data point $\mathbf{x}^{(i))}$). In most RBF types, $\boldsymbol{\varphi}$ is a function of the shape parameter, that is, $\boldsymbol{\varphi}=\boldsymbol{\varphi}(\mathbf{x};\theta)$. However, for the sake of simplicity, we eliminate the dependency of $\boldsymbol{\varphi}$ on $\theta$ when writing down the basis function

The coefficients $\boldsymbol{\alpha}$ are calculated by least squares. To be more exact, consider the following linear system



> $\mathbf{K}\boldsymbol{\alpha}=\mathbf{y}$



where $\mathbf{K}$ is the Gram Matrix with its $(i,j)$ component is $\mathbf{K}_{i,j} = \varphi(\|\mathbf{x}^{(i)}-\mathbf{x}^{(j)}\|)$. The coefficients can then be simply computed by

> $\boldsymbol{\alpha}=\mathbf{K}^{-1}\mathbf{y}$


## Kernel function
One important building block of an RBF model is the **Kernel function**, or simply the basis function. In this tutorial, we will use three popular RBFs, namely: 

*   Gaussian
> $\text{exp}\big(-\frac{\|r\|^{2}}{\theta} \big)$

*   Multiquadric
> $\big(1+\frac{\|r\|^{2}}{\theta} \big)^{1/2}$
*   Inverse multiquadric
> $\big(1+\frac{\|r\|^{2}}{\theta} \big)^{-1/2}$



In [ ]:
def Kernel_RBF(
    sigma, dist, kernel_type, shape_type
):
  if shape_type in 'Isotropic':

    if kernel_type in 'Gaussian' :
      kernel = np.exp((-1*dist**2)/sigma)
    elif kernel_type in 'invmultiquadric' :
      kernel = 1/(1+sigma*dist**2)
    elif kernel_type in 'multiquadric' :
      kernel = np.sqrt(1+sigma*dist**2)
    else:
      print("Kernel is not recognized!")

  elif shape_type in 'Anisotropic':

    if kernel_type in 'Gaussian' :
      kernel = np.exp(np.sum(-1*dist**2/sigma))
    elif kernel_type in 'invmultiquadric' :
      kernel = 1/np.sum(1+sigma*dist**2)
    elif kernel_type in 'multiquadric' :
      kernel = np.sqrt(np.sum(1+sigma*dist**2))
    else:
      print("Kernel is not recognized!")

  kernel = np.array([kernel])
  return kernel

## Calculation of RBF coefficients

With a given $\theta$ and kernel type, we can construct the Gram matrix $\mathbf{K}$ based on the experimental design $\mathbf{X}$ and the corresponding responses $\mathbf{y}$. The next step is to calculate $\boldsymbol{\alpha}$ by solving $\boldsymbol{\alpha}=\mathbf{K}^{-1}\mathbf{y}$. The following cell does the job:

In [ ]:
def model_RBF(
    sigma, X, y, bounds, kernel_type, shape_type
):

  Nsamp = X.shape[0]
  dimen = X.shape[1]

  #normalize X
  Xnorm = np.zeros(shape = [Nsamp, dimen])
  for i in range(dimen):
     Xnorm[:, i] = (X[:, i] - bounds[0, i])/(bounds[1, i] - bounds[0, i])

  # Construct kernel matrix
  Phi = np.zeros(shape=[Nsamp, Nsamp])

  for i in range(Nsamp):
     for j in range(Nsamp):
       dist = np.sqrt(np.sum((Xnorm[i, :] - Xnorm[j, :])**2))
       dist = np.array([dist])
       Phi[i, j] = Kernel_RBF(sigma, dist, kernel_type, shape_type)

  w = np.linalg.solve(Phi, y)

  return w, Phi

def check_posdef(Phi):
  if np.all(np.linalg.eigvals(Phi) > 0):
    check = 1
  else:
    check = 0

  return check

## RBF prediction
After the RBF model has been constructed (i.e., obtaining $\boldsymbol{\alpha}$), the calculation of RBF prediction is straightforward. The following cell builds the function for RBF prediction:


In [ ]:
def Pred_RBF(
    xpred, sigma, X, w, bounds, kernel_type, shape_type
):
  dimen = X.shape[1]
  Nsamp = X.shape[0]
  Npred = xpred.shape[0]
  norm_pred = np.zeros(shape=[Npred, dimen])
  Xnorm = np.zeros(shape = [Nsamp, dimen])
  for i in range(dimen):
    norm_pred[:, i] = (xpred[:, i]- bounds[0, i])/(bounds[1, i]- bounds[0, i])
    Xnorm[:, i] = (X[:, i] - bounds[0, i])/(bounds[1, i] - bounds[0, i])

  
  #contruct prediction rbf vector
  Npred = xpred.shape[0]
  
  pred_phi = np.zeros(shape = [Npred, Nsamp])

  for j in range(Npred):
    for i in range(Nsamp):
      dist = np.sqrt(np.sum((Xnorm[i, :] - norm_pred[j, :])**2))
      dist = np.array([dist])
      pred_phi[j, i] = Kernel_RBF(sigma, dist, kernel_type, shape_type)

  ypred = np.zeros(shape=[Npred])
  
  for i in range(Npred):
    ypred[i] = pred_phi[i, :] @ w

  return ypred

## LOOCV-based shape parameter optimization
The shape parameter in RBF is usually optimized by minimizing the LOOCV error. There exists an analytical formulation to calculate the LOOCV error in an RBF model, reads as

$\hat{f}_{i}(\mathbf{x}^{(i)})-y^{(i)} = \frac{\alpha_{i}}{\mathbf{K}_{ii}}$,

which can be used for any error metric (e.g. RMSE). However, because this tutorial is given for educational purpose, we will do it the hard way. That is, we will build $n$ extra RBF models to calculate the LOOCV error. However, in practive, note that we always use the analytical formulation to speed up the computation time.

The following cell defines the subroutine for LOOCV calculation:


In [ ]:
def LOOCV_RBF(
    sigma, X, y, bounds, kernel_type, shape_type
):

  Nsamp = X.shape[0]
  Ypred = np.zeros(shape=[Nsamp])

  for i in range(Nsamp):
    Xpred = X[i, :].reshape(1, dimen)
    Xtemp = np.delete(X, i, 0)
    Ytemp = np.delete(y, i)

    wtemp, Phitemp = model_RBF(sigma, Xtemp, Ytemp, bounds, kernel_type, shape_type)
    check = check_posdef(Phitemp)

    if check == 0:
      break

    Ypred[i] = Pred_RBF(Xpred, sigma, Xtemp, wtemp, bounds, kernel_type, shape_type)
  
  if check == 1:
    rmse = np.sqrt(np.sum((Ypred - y)**2)/Nsamp)
    rmse = np.array([rmse])
  elif check == 0:
    rmse = 1e9
    
  return rmse

## Main RBF routine
Finally, let's combine all subroutines into an integrated code:

In [ ]:
def main_RBF(
    X, y, bounds, kernel_type = 'Gaussian', shape_type = 'Isotropic'
):

  class get_result:
    def __init__ (self):
      self.X = X
      self.y = y
      self.bounds = bounds
      self.kernel_type = kernel_type
      self.shape_type = shape_type
      self.LOOCV = rmse
      self.weight = w
      self.sigma = sigma
      self.Phi = Phi

  dimen = X.shape[1]
  if shape_type in 'Anisotropic':
    sigma_init = np.ones(shape = [dimen])
  else:
    sigma_init = np.array([1])

  params = (X, y, bounds, kernel_type, shape_type)
  opt_param = optimize.minimize(LOOCV_RBF, sigma_init, args=params, method='BFGS')
  sigma = opt_param.x
  
  rmse = LOOCV_RBF(sigma, X, y, bounds, kernel_type, shape_type)

  w, Phi = model_RBF(sigma, X, y, bounds, kernel_type, shape_type)

  return get_result()

## Example RBF1: Application to a one-dimensional function

Our first test case is the Forrester's function, defined as

$f(x)=(6x-2)^2\text{sin}(12x-4)$

In the cell below, you change the ```kernel_type``` and the number of samples (```Nsamp```) to see how it would affect the RBF prediction. The shape parameter is automatically optimized by using BFGS method.

In [ ]:
# Create the sampling points
dimen = 1 # Problem's dimensionality
Nsamp = 10 # Number of samples, you can change this and see how it would affect the surrogate model
ub = np.array([0]) # Upper bound
lb = np.array([1]) # Lower bound
bounds = np.array([lb, ub]) # Upper and lower bound

X = np.linspace(0,1,num=Nsamp,endpoint=True).reshape(-1,1)
y = (6*X-2)**2 *np.sin(12*X-4)
y = y.reshape(-1,1)

numplot = 50
Xpred = np.linspace(0, 1, numplot).reshape(-1,1) # Prediction sites
yreal = (6*Xpred-2)**2 *np.sin(12*Xpred-4) # True values at the prediction site

# Construct the RBF model (notice that you can change the kernel function)
myRBF = main_RBF(X, y, bounds,kernel_type = 'Gaussian') # Construct RBF
ypred = Pred_RBF(Xpred, myRBF.sigma, myRBF.X, myRBF.weight, myRBF.bounds, myRBF.kernel_type, myRBF.shape_type) # Create prediction

# Plotting
plt.plot(Xpred, yreal, 'kx-') #
plt.plot(Xpred, ypred, 'bd')
plt.scatter(X,y,s=100,c='r')
plt.xlabel('$x_{1}$')
plt.ylabel('$y$')
plt.legend(['True','Prediction','Sampling points'])
plt.show()

---
## Example RBF2: Application to a two-dimensional function
The next problem is a two-dimensional function, reads as
$f(\mathbf{x})= x_{1}^{3}+x_{1}+x_{2}^{2}-x_{2}$:

Let's try our RBF model on that function. The first step is to generate the sampling points. After that, we can build our two-dimensional RBF model:

In [ ]:
#2 dimensional example
dimen = 2
Nsamp = 30

lb = np.array([-2, -2]) # Lower bounds
ub = np.array([2, 2]) # Upper bounds
bounds = np.array([lb, ub]) # Lower and upper bounds

X = sampling_rlh(Nsamp, dimen) # Random latin hypercube sampling
X = X*(bounds[1, :] - bounds[0, :]) + bounds[0, :] # Convert into the real scale
y = fun2D_polynom(X) # Evaluate the function

After that, we can build our two-dimensional RBF model:

In [ ]:
# Construct the RBF model 
myRBF2 = main_RBF(X, y, bounds, kernel_type = 'Gaussian') # Construct RBF

Let's plot the results:

In [ ]:
#@title RBF prediction on a two-dimensional function
# For plotting purpose
numplot = 50
Xpred = np.linspace(lb[0], ub[0], numplot)
Ypred = np.linspace(lb[1], ub[1], numplot)

Xplot, Yplot = np.meshgrid(Xpred, Ypred)
Zplot = np.zeros(shape=[numplot, numplot])
XYpred = np.zeros(shape=[numplot**2, dimen])

c = 0
for i in range(numplot):
  for j in range(numplot):
    XYpred[c, 0] = Xpred[i]
    XYpred[c, 1] = Ypred[j]
    Zplot[i, j] = fun2D_polynom(XYpred[c, :].reshape(1, dimen))
    c = c+1

# Plotting the RBF prediction
ypred = Pred_RBF(XYpred, myRBF2.sigma, myRBF2.X, myRBF2.weight, myRBF2.bounds, myRBF2.kernel_type, myRBF2.shape_type) # Create prediction
Zpred = np.zeros(shape = [numplot, numplot])

c= 0
for i in range(numplot):
  for j in range(numplot):
    Zpred[i, j] = ypred[c]
    c = c+1

plt.figure(0)
cf = plt.contourf(Xplot, Yplot, Zplot)
plt.colorbar()
plt.scatter(X[:,0],X[:,1],s=60,c='r')
plt.title("Real Surface")

plt.figure(1)
cf = plt.contourf(Xplot, Yplot, Zpred)
plt.colorbar()
plt.scatter(X[:,0],X[:,1],s=60,c='r')
plt.title("RBF Surface")


---
## Example RBF3: Application to the 10-dimensional Wing Weight Function
The next example is more difficult due to its higher dimensionality. The function that we use is the so-called wing-weight function (see [here](https://www.sfu.ca/~ssurjano/wingweight.html) for more details). Because we use a traditional method for LOOCV calculation, the overall model building time is relatively time consuming. 

The following cell defines the wing-weight function:

In [ ]:
def wingweight(
    X
):

  Nsamp = X.shape[0]
  X[:, 3] = X[:, 3] * np.pi / 180
  y = 0.036 * X[:, 0]**(0.758) * X[:, 1]**(0.0035) * (X[:, 2]/(np.cos(X[:, 3])**2))**(0.6) * X[:, 4]**(0.006) * X[:, 5]**(0.04) * (100*X[:, 6]/(np.cos(X[:,3])))**(-0.3) * X[:, 7] * X[:, 8] + X[:, 1] * X[:, 9]
  y = y.reshape(Nsamp)

  return y

The following cell builds the sampling points and also the evaluation data set:

In [ ]:
# 10 dimensional function
dimen = 10

lb = np.array([150, 220, 6, -10, 16, 0.5, 0.08, 2.5, 1700, 0.025]) # Lower bounds
ub = np.array([200, 300, 10, 10, 45, 1, 0.18, 6, 2500, 0.08]) # Upper bounds
bounds = np.array([lb, ub]) # Lower and upper bounds

# for evaluation data set
Nval = 1000 # Size of validation samples
Xval = sampling_rlh(Nval, dimen) # Validation samples
Xval = Xval*(bounds[1, :] - bounds[0, :]) + bounds[0, :] #Convert to real scale
yval = wingweight(Xval) # Responses at validation samples

# build the DoE
Nsamp = 30  # Sample size

X = sampling_rlh(Nsamp, dimen) # Sampling points
X = X*(bounds[1, :] - bounds[0, :]) + bounds[0, :] #Convert to real scale
y = wingweight(X) # Responses at sampling points

Finally, the following cell builds our RBF model to approximate the 10-dimensional wing weight function (we have to wait for a while because we use a conventional LOOCV for shape parameter optimization).

In [ ]:
# RBF model building
myRBF3 = main_RBF(X, y, bounds, kernel_type='Gaussian', shape_type='Isotropic')


Since we cannot plot a ten-dimensional function, we can assess the accuracy by calculating the root-mean-squared-error on the validation dataset. We also normalize the error by the mean of validation samples so as to allow easier interpretation.

In [ ]:
ypred = Pred_RBF(Xval, myRBF3.sigma, myRBF3.X, myRBF3.weight, myRBF3.bounds, myRBF3.kernel_type, myRBF3.shape_type) # Create prediction

# calculate true error
err = np.sqrt(np.sum((yval-ypred)**2)/Nval)
true_err = err/np.mean(yval)
print("Normalized RMSE:")
print(true_err)

---
# Application of Surrogate Models: Optimization of Branin function
Reference: https://www.sfu.ca/~ssurjano/branin.html

In the last part of this tutorial, we are going to implement PR and RBF to estimate the global optimum of a two-dimensional function. The function that we are going to optimize is the Branin function, which is defined as follows:

$ f(\mathbf{x}) = a (x_2 - bx_1^2 + cx_1 - r)^2 + s(1-t) cos(x_1) + s $

where $a = 1, b = 5.1/(4\pi^2), c = 5/\pi, r = 6, s = 10, t = 1/(8\pi) $.

The branin function, or also known as Branin-Hoo function, has three global minima. The function is usually evaluated at $x_1 \in [-5, 10], x_2 \in [0, 15] $. The function is coded below:

In [ ]:
#@title Branin function
def Branin(
    x, dimen = 2
):
  x_shape = x.shape
  ns = x_shape[0]

  f = np.zeros(shape=[ns]) 
  a = 1
  b = 5.1/(4*np.pi**2)
  c = 5/np.pi
  r = 6
  s = 10
  t = 1/(8*np.pi)
  for i in range(ns):  
    f[i] = a*(x[i,1]-b*x[i,0]**2+c*x[i,0]-r)**2 + s*(1-t)*np.cos(x[i,0]) + s
 
  return f

Now, we are going to perform an optimization process aided by surrogate models. What we are going to do first is to create the surrogate model and then treat  the prediction as the objective function in the optimization process.

First, let's set up the required dataset:

In [ ]:
dimen = 2 #Dimension of the problem

lb = np.array([-5, 0]) # Lower bounds
ub = np.array([10, 15]) # Upper bounds
bounds = np.array([lb, ub]) # Lower and upper bounds

# build the DoE
Nsamp = 40  # Sample size

X = sampling_rlh(Nsamp, dimen) # Sampling points
X = X*(bounds[1, :] - bounds[0, :]) + bounds[0, :] #Convert to real scale
y = Branin(X) # Responses at sampling points

And now we are going to make both polynomial regression model and radial basis function model.

In [ ]:
# Construct the RBF model 
myRBF_opt = main_RBF(X, y, bounds, kernel_type = 'invmultiquadric', shape_type = 'Isotropic')
myPCE_opt = main_PR(X, y, bounds, max_order = 4, poly_type = 'legendre')

What we need to do next is to define the objective function code. The following will do the job:

In [ ]:
## Objective function for RBF model
def Obj_RBF(Xpred, output_RBF):
  ypred = Pred_RBF(Xpred.reshape(1,-1), output_RBF.sigma, output_RBF.X, output_RBF.weight, output_RBF.bounds, output_RBF.kernel_type, output_RBF.shape_type)

  return ypred

def Obj_PCE(Xpred, output_PCE):
  ypred = pred_PR(Xpred.reshape(1,-1), output_PCE.coef, output_PCE.Id_pred, output_PCE.poly_type)
  return ypred

Let's try to apply the two surrogate models for actual optimization process.
Alright, all that's left is doing the actual optimization part. We are going to start at point $\mathbf{x} = (0, 0)$ so that the point is not too far to the global minimum. We are going to use gradient-based method for the optimization process. Do note that using surrogate model does not prevent your optimization steps being stuck at local optimum.

In [ ]:
## Optimization for RBF model
x_init = np.array([0,0])

opt_RBF = optimize.minimize(Obj_RBF, x_init, args = (myRBF_opt), method = 'BFGS')

## Optimization for PCE model
opt_PCE = optimize.minimize(Obj_PCE, x_init, args = (myPCE_opt), method = 'BFGS')

print("Optimum X from RBF model:")
print(opt_RBF.x)
print("Optimum value from RBF model:")
print(opt_RBF.fun)
print("Optimum X from PCE model:")
print(opt_PCE.x)
print("Optimum value from PCE model:")
print(opt_PCE.fun)

According to references, the three global minima are located at $ \mathbf{x}^{*} = (-\pi, 12.275); (\pi, 2.275); (9.42478, 2.475) $ and $f(\mathbf{x}^*) = 0.397887 $. The nearest local optimum with our initial point is located at $(\pi, 2.275) $. In practice, there are further methods to help tackling this problem, such as Expected Improvement.

In [ ]:
# For plotting purpose
numplot = 50
Xpred = np.linspace(lb[0], ub[0], numplot)
Ypred = np.linspace(lb[1], ub[1], numplot)

Xplot, Yplot = np.meshgrid(Xpred, Ypred)
Zplot = np.zeros(shape=[numplot, numplot])
XYpred = np.zeros(shape=[numplot**2, dimen])
c = 0

for i in range(numplot):
  for j in range(numplot):
    XYpred[c, 0] = Xpred[i]
    XYpred[c, 1] = Ypred[j]
    Zplot[i, j] = Branin(XYpred[c, :].reshape(1, dimen))
    c = c+1

# Plotting the RBF prediction
Zpred_RBF = np.zeros(shape = [numplot, numplot])
Zpred_PCE = np.zeros(shape = [numplot, numplot])

c= 0
for i in range(numplot):
  for j in range(numplot):
    Zpred_RBF[i, j] = Obj_RBF(XYpred[c, :], myRBF_opt)
    Zpred_PCE[i, j] = Obj_PCE(XYpred[c, :], myPCE_opt)
    c = c+1

lb_plot = -40
ub_plot = 320
levels_color = np.linspace(lb_plot, ub_plot, 9)

plt.figure(0)
cf = plt.contourf(Xplot, Yplot, Zplot, levels =levels_color)
plt.colorbar()
plt.clim(lb_plot, ub_plot)
plt.scatter(X[:,0],X[:,1],s=60,c='r')
plt.title("Real Surface")

plt.figure(1)
cf = plt.contourf(Xplot, Yplot, Zpred_RBF, levels = levels_color)
plt.colorbar()
plt.clim(lb_plot, ub_plot)
plt.scatter(X[:,0],X[:,1],s=60,c='r')
plt.title("RBF Surface")

plt.figure(2)
cf = plt.contourf(Xplot, Yplot, Zpred_PCE, levels = levels_color)
plt.colorbar()
plt.clim(lb_plot, ub_plot)
plt.scatter(X[:, 0], X[:, 1], s=60, c='r')
plt.title("PCE Surface")

While the optimum prediction is not really that accurate, the overall trend of the function is well-approximated.

In [ ]:
Branin_RBF_err = np.zeros(shape=[numplot**2])
Branin_PCE_err = np.zeros(shape = [numplot**2])
c = 0
for i in range(numplot):
  for j in range(numplot):
    Branin_RBF_err[c] = (Zplot[i,j] - Zpred_RBF[i,j])**2
    Branin_PCE_err[c] = (Zplot[i,j] - Zpred_PCE[i,j])**2

mean_val = np.sum(np.sum(Zplot))/numplot**2
Branin_RBF_rmse = np.sqrt(np.sum(Branin_RBF_err)/numplot**2)/mean_val
Branin_PCE_rmse = np.sqrt(np.sum(Branin_PCE_err)/numplot**2)/mean_val

print("RMSE RBF: ")
print(Branin_RBF_rmse)
print("RMSE PCE: ")
print(Branin_PCE_rmse)

---
# Afterword

The main point of using surrogate model is to "replace" our functions of interest. Most researchs about surrogate modelling are motivated by the fact that some important functions are either expensive to evaluate or do not have necessary information such as derivatives. Besides optimization, surrogate modelling are also used for reliability analysis, sensitivity analysis, etc. Some surrogate models are more suited to handle spesific problems than others. Big data is the realm of deep neural network, small to medium engineering data is the domain of GPR/RBF, small to medium general regression data is the domain of gradient boosting, and UQ is the domain of PCE.
